In [5]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/data-analyst-job-roles-in-canada/Raw_Dataset.csv
/kaggle/input/data-analyst-job-roles-in-canada/Cleaned_Dataset.csv


In [3]:
import pandas as pd
import numpy as np
import re

df = pd.read_csv(RAW_PATH)

# 1) 컬럼명 통일
df.columns = [c.strip().lower().replace(" ","_") for c in df.columns]

# 2) 중복 제거
df = df.drop_duplicates(subset=['job_title','company_name','city','province'])

# 3) 결측치 처리 (도시, 주 없는 건 제거)
df = df.dropna(subset=['city','province'])

# 4) 급여 정리
def parse_salary(s):
    if pd.isna(s): return (np.nan,np.nan)
    t = str(s).lower().replace(",","")
    t = re.sub(r'cad|\$','',t)
    t = re.sub(r'(\d+)k', lambda m: str(int(m.group(1))*1000), t)
    m = re.findall(r'(\d+)\s*[-–]\s*(\d+)', t)
    if m:
        return (float(m[0][0]), float(m[0][1]))
    m2 = re.findall(r'(\d+)', t)
    if m2:
        return (float(m2[0]), float(m2[0]))
    return (np.nan,np.nan)

df[['salary_min','salary_max']] = df['job_salary'].apply(parse_salary).apply(pd.Series)
df['salary_mid'] = df[['salary_min','salary_max']].mean(axis=1)

# 5) 기술스택 파싱 (옵션)
skills = ['sql','python','excel','tableau','power bi']
for skill in skills:
    df[skill.replace(" ","_")] = df['language_and_tools'].str.contains(skill, case=False, na=False).astype(int)

# 저장
df.to_csv(OUT_PATH, index=False)
print("Saved:", OUT_PATH)


Saved: /kaggle/working/Cleaned_Dataset_Sopia.csv


I attempted to clean the raw dataset, but due to encoding issues in the salary column, I proceeded with the cleaned dataset for the main analysis. Both versions are included for transparency.

In [2]:
import re
import numpy as np
import pandas as pd

RAW_PATH = "/kaggle/input/data-analyst-job-roles-in-canada/Raw_Dataset.csv"
OUT_PATH = "/kaggle/working/Cleaned_Dataset_Sopia.csv"

df = pd.read_csv(RAW_PATH)
print(df.columns.tolist())


['Job ID', 'Job Title', 'Company Name', 'Language and Tools', 'Job Salary', 'City', 'Province', 'Job Link']
